In [188]:
import pandas as pd
data = pd.read_csv("Chocolate Sales.csv")
data['Amount'] = data['Amount'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)
data.head(2)

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04/01/2022,5320.0,180
1,Van Tuxwell,India,85% Dark Bars,01/08/2022,7896.0,94


In [189]:
features = [ 'Country', 'Product', 'Boxes Shipped']
Y = data.Amount
X = data[features]

In [190]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=0.8, test_size=0.2)

In [191]:
categorical = [i for i in X_train.columns if X_train[i].nunique() < 25 and X_train[i].dtype == 'object']
numerical = [i for i in X_train.columns if X_train[i].dtype in ['int64', 'float64']]


In [192]:
my_cols = categorical+numerical
my_cols

['Country', 'Product', 'Boxes Shipped']

In [193]:
s = (X_train.dtypes == 'object')
obj = list(s[s].index)

In [194]:
from sklearn.preprocessing import OneHotEncoder
OH = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_X_train = pd.DataFrame(OH.fit_transform(X_train[obj]))
OH_X_test = pd.DataFrame(OH.transform(X_test[obj]))

In [195]:
OH_X_train.index = X_train.index
OH_X_test.index = X_test.index

In [196]:
num_X_train = X_train.drop(obj, axis =1)
num_X_test = X_test.drop(obj, axis =1)

In [197]:
OH_X_train = pd.concat([OH_X_train, num_X_train], axis=1)
OH_X_test = pd.concat([OH_X_test, num_X_test], axis=1)

In [198]:
OH_X_train.columns = OH_X_train.columns.astype('str')
OH_X_test.columns = OH_X_test.columns.astype('str')

In [204]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
model = RandomForestRegressor(n_estimators=100 , random_state=0)
model.fit(OH_X_train, Y_train)
pred = model.predict(OH_X_test)
mae = mean_absolute_error(Y_test, pred)
price = Y.mean()
error = (mae/price)*100
print(f"MAE: {mae}")
print(f"Average Prices: {price}")
print(f"Error rate: {error}")

MAE: 2299.4954659307746
Average Prices: 6030.338775137113
Error rate: 38.132110842785785
